In [23]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [24]:
# 우리가 사용할 컴퓨터를 check하는 부분
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [25]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# 1단계 - 물리적인 위치에 데이터 저장...다운로드(training/test)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(), 
                                           download=True)
# training에 ToTensor()를 통해서 스케일링 해준 것처럼 test도 동일한 작업을 해준다.
test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=False, 
                                           transform=transforms.ToTensor())

# 2단계 - 네트워크에 100개씩 잘라서 로드함
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)

### Convolution NeuralNet Model 생성

In [26]:
class ConvNet(nn.Module):
    
    
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, # in_channels...흑백 : 1
                      16, # out_channels
                      kernel_size=5, # filter size
                      stride=1, 
                      padding=2),
            nn.ReLU() # activate fct.
        )
        self.layer2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), 
            nn.ReLU()
        )
        self.layer4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer5 = nn.Linear(7*7*32, num_classes) # 최종 결과를 낼 때는 FCN
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.layer5(out)

        return out

In [27]:
model = ConvNet(num_classes).to(device)

In [28]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [29]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (i+1) % 100 == 0:
       print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [600/600], Loss: 0.1813
Epoch [2/5], Step [600/600], Loss: 0.0239
Epoch [3/5], Step [600/600], Loss: 0.0188
Epoch [4/5], Step [600/600], Loss: 0.0381
Epoch [5/5], Step [600/600], Loss: 0.0017


In [30]:
model.eval()
with torch.no_grad(): # 실제로 학습할 필요가 없으면 no.grad()구문을 넣어준다.
    correct = 0
    total = 0
    for images, labels in test_loader: # 인덱스를 굳이 안받아도 되니깐 enumeratorr가 빠졌다.
        images = images.to(device) #한줄로 죽 펼친 다음에 입력받아서
        labels = labels.to(device) # 라벨도 마찬가지로
        outputs = model(images) #모델에 넣고
        _, predicted = torch.max(outputs.data, 1)
        # total은 푼 문제, correct는 맞춘 문제수..이런식으로 카운팅이 들어간다
     
        total += labels.size(0) 
        correct += (predicted == labels).sum().item()
        print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 100.0 %
Test Accuracy of the model on the 10000 test images: 100.0 %
Test Accuracy of the model on the 10000 test images: 100.0 %
Test Accuracy of the model on the 10000 test images: 99.75 %
Test Accuracy of the model on the 10000 test images: 99.6 %
Test Accuracy of the model on the 10000 test images: 99.5 %
Test Accuracy of the model on the 10000 test images: 99.28571428571429 %
Test Accuracy of the model on the 10000 test images: 99.125 %
Test Accuracy of the model on the 10000 test images: 99.11111111111111 %
Test Accuracy of the model on the 10000 test images: 99.0 %
Test Accuracy of the model on the 10000 test images: 98.9090909090909 %
Test Accuracy of the model on the 10000 test images: 99.0 %
Test Accuracy of the model on the 10000 test images: 98.84615384615384 %
Test Accuracy of the model on the 10000 test images: 98.78571428571429 %
Test Accuracy of the model on the 10000 test images: 98.8 %
Test Accuracy of the model on 